In [94]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [95]:
import sys
import os
import h5py
import scipy.io
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import Input
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json

In [96]:
############################################################################################
# Nuclei_feature
############################################################################################
nuc_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/nuc_Feature/'
nuc_file_name = nuc_dir+ 'bach_mp_nuclei_feature.mat'
nuc_data = []
with h5py.File(nuc_file_name) as f:
    for column in f['bach_mp_nuclei_feature']:
        row_data = []
        for row_number in range(len(column)):            
            row_data.append(f[column[row_number]][:])   
        nuc_data.append(row_data)
        
Nuc_feat = np.array(nuc_data)
del nuc_data

In [97]:
############################################################################################
# Cascaded PCA
############################################################################################
pca_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/PCA_Feature/'
pca_file_name = pca_dir+ 'bach_pca_feature.mat'
pca_data = []
with h5py.File(pca_file_name) as f:
    for column in f['bach_pca_feature']:
        pca_data.append(column)
        
pca_data = np.array(pca_data)

In [98]:
############################################################################################
# CNN feature
############################################################################################
vgg_train = np.load('fine_bottleneck_feat_BACH.npy')

In [72]:
print(Nuc_feat.shape)
print(pca_data.shape)
print(vgg_train.shape)


(400, 1, 256, 256)
(400, 12)
(400, 7, 7, 512)


In [99]:
Nuc_feat_n = Nuc_feat/((Nuc_feat-Nuc_feat.min()).max())
pca_data_n = pca_data/((pca_data-pca_data.min()).max())
vgg_train_n = vgg_train/((vgg_train-vgg_train.min()).max())

In [109]:
mag_fact = '200'
model_new_name = 'Final_2_concat_Model_' + mag_fact
# load json and create model
json_file = open(model_new_name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_2 = model_from_json(loaded_model_json)
# load weights into new model
loaded_model_2.load_weights(model_new_name + '.h5')
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model_2.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
loaded_model_2.summary()

Loaded model from disk
Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_11_input (InputLayer)    (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 254, 254, 32) 320         conv2d_11_input[0][0]            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 252, 252, 64) 18496       conv2d_11[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 126, 126, 64) 0           conv2d_12[0][0]                  
____________________________________________________________________

In [100]:
Nuc_feat = Nuc_feat.reshape(len(Nuc_feat), 256,256,1)
vgg_train_reshape = vgg_train.reshape(400, 7*7*512)


Nuc_feat_n = Nuc_feat/((Nuc_feat-Nuc_feat.min()).max())
pca_data_n = pca_data/((pca_data-pca_data.min()).max())
vgg_train_n = vgg_train_reshape/((vgg_train_reshape-vgg_train_reshape.min()).max())


In [102]:
train_labels = np.array([0] * 200 + [1] * 200)

In [111]:
score = loaded_model_2.evaluate([Nuc_feat, vgg_train_reshape],train_labels)
#score = loaded_model_2.evaluate([Nuc_feat_n, pca_data_n, vgg_train_n],train_labels)
print('\n', 'Test accuracy:', score[1])

400/400 [==============================] - 13s 33ms/step

 Test accuracy: 0.5


In [107]:
pred = loaded_model_2.predict([Nuc_feat_n, pca_data_n, vgg_train_n])

In [108]:
print(pred)

[[0.74930865]
 [0.78814113]
 [0.810473  ]
 [0.75409245]
 [0.8350094 ]
 [0.79666406]
 [0.79702353]
 [0.80045426]
 [0.7834956 ]
 [0.7499637 ]
 [0.81221426]
 [0.7550408 ]
 [0.75239456]
 [0.7696227 ]
 [0.8051413 ]
 [0.79063445]
 [0.8075762 ]
 [0.7513358 ]
 [0.7943564 ]
 [0.79099125]
 [0.76704633]
 [0.811014  ]
 [0.8108218 ]
 [0.75499517]
 [0.75974095]
 [0.78302646]
 [0.77344316]
 [0.7954647 ]
 [0.83052844]
 [0.7506574 ]
 [0.79812956]
 [0.7648085 ]
 [0.7609067 ]
 [0.767153  ]
 [0.7592447 ]
 [0.7842914 ]
 [0.7957019 ]
 [0.78412366]
 [0.8278085 ]
 [0.7843589 ]
 [0.7728967 ]
 [0.781503  ]
 [0.8014039 ]
 [0.7947039 ]
 [0.81717384]
 [0.7748356 ]
 [0.80042833]
 [0.79689944]
 [0.8205786 ]
 [0.7729983 ]
 [0.81389785]
 [0.7980165 ]
 [0.7808332 ]
 [0.81148535]
 [0.8205223 ]
 [0.7549127 ]
 [0.7341942 ]
 [0.77038795]
 [0.75059676]
 [0.77538985]
 [0.8173379 ]
 [0.8419097 ]
 [0.7801584 ]
 [0.80397826]
 [0.78888637]
 [0.79329705]
 [0.7958514 ]
 [0.76869947]
 [0.7831737 ]
 [0.79332024]
 [0.775832  ]
 [0.78